In [3]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm

FILE_OUT_NAME = "flats_photos.csv"

link_base = "https://www.house.kg/kupit-kvartiru?page="

columns = ["link", "photo_url"]

data = {i: [] for i in columns}

left_border = int(input("From which page do you want to parse: "))
right_border = int(input("Until which page do you want to parse: "))  # 651

for i in tqdm(range(left_border, right_border + 1), desc='Парсинг страниц'):
    link = link_base + str(i)
    page_req = requests.get(link)
    page_soup = BeautifulSoup(page_req.text, "html.parser")
    page_table = page_soup.find("div", attrs={"class": "listings-wrapper"})

    if page_table is None:
        continue

    flats = page_table.find_all("div",
                                attrs={"itemtype": "https://schema.org/Apartment", "class": "listing"})
    for flat in flats:
        flat_link = "https://www.house.kg" + flat.find("a")["href"]
        
        # Запрос страницы конкретного объявления
        flat_req = requests.get(flat_link)
        flat_soup = BeautifulSoup(flat_req.text, "html.parser")

        # Поиск всех фотографий
        photo_tags = flat_soup.find_all("img", attrs={"class": "temp-auto lazy"})  # Важно заменить на правильный класс

        if photo_tags:
            for photo in photo_tags:
                photo_url = photo.get("src")
                if photo_url:
                    data["link"].append(flat_link)
                    data["photo_url"].append(photo_url)

# Создание DataFrame с ссылками на фотографии
df = pd.DataFrame(data)
df.to_csv(FILE_OUT_NAME, index=False)
print('End of program')


Парсинг страниц: 100%|██████████| 9/9 [01:57<00:00, 13.06s/it]

End of program
